In [1]:
# import libraries

import requests
import pandas as pd
import numpy as np

In [2]:
# use FPL API to access FPL data

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
# create dataframes for teams, elements, and element_type data

teams_df = pd.DataFrame(json['teams'])

elements_df = pd.DataFrame(json['elements'])

elements_types_df = pd.DataFrame(json['element_types'])


In [4]:
# filter out irrelevent columns from elements_df

slim_elements_df = elements_df[['id','first_name','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points', 'status']]

# map position from element_types_df into slim_elements_df

slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

# map team name from teams_df into slim_elements_df

slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

# ensure all value info is of type int

slim_elements_df['value_season'] = slim_elements_df.value_season.astype(float)

# add points per minute

slim_elements_df['ppg'] = (slim_elements_df['total_points'] / slim_elements_df['minutes']) * 90

# remove players with less than 18 games played minutes

slim_elements_df = slim_elements_df.loc[slim_elements_df.minutes > 0]

# add new value metric: points per minute per cost

slim_elements_df['value'] = slim_elements_df['ppg'] / slim_elements_df['now_cost']

C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [5]:
# create new dataframes for each position

fwd_df = slim_elements_df.loc[slim_elements_df.position == 'Forward']
mid_df = slim_elements_df.loc[slim_elements_df.position == 'Midfielder']
def_df = slim_elements_df.loc[slim_elements_df.position == 'Defender']
goal_df = slim_elements_df.loc[slim_elements_df.position == 'Goalkeeper']

In [11]:
# create new dataframes for cheap players in each position

cheap_fwd_df = fwd_df[fwd_df['now_cost'] <= fwd_df['now_cost'].min()+5]
cheap_mid_df = mid_df[mid_df['now_cost'] <= mid_df['now_cost'].min()+5]
cheap_def_df = def_df[def_df['now_cost'] <= def_df['now_cost'].min()+5]
cheap_goal_df = goal_df[goal_df['now_cost'] <= goal_df['now_cost'].min()+5]

In [27]:
cheap_fwd_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
50,49,Keinan,Davis,Aston Villa,4,3.2,45,265,0,5.1,23,d,Forward,7.811321,0.173585
401,344,Daniel,N'Lundulu,Southampton,4,0.1,45,102,0,2.9,13,u,Forward,11.470588,0.254902
272,235,Divock,Origi,Liverpool,4,1.3,50,180,0,2.0,10,a,Forward,5.000000,0.100000
405,348,Michael,Obafemi,Southampton,4,12.7,45,57,0,0.9,4,a,Forward,6.315789,0.140351
528,523,Patrick,Cutrone,Wolves,4,0.2,50,23,0,0.4,2,a,Forward,7.826087,0.156522
442,520,Dane,Scarlett,Spurs,4,1.2,45,1,0,0.2,1,a,Forward,90.000000,2.000000


In [26]:
cheap_mid_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
428,365,Pierre-Emile,Højbjerg,Spurs,3,3.1,50,3420,0,21.4,107,a,Midfielder,2.815789,0.056316
495,421,Declan,Rice,West Ham,3,4.5,50,2880,0,17.2,86,a,Midfielder,2.687500,0.053750
228,216,Wilfred,Ndidi,Leicester,3,1.6,50,2175,0,15.8,79,a,Midfielder,3.268966,0.065379
511,434,Leander,Dendoncker,Wolves,3,0.2,50,2498,0,15.4,77,a,Midfielder,2.774219,0.055484
103,70,Yves,Bissouma,Brighton,3,13.4,45,3112,0,17.1,77,a,Midfielder,2.226864,0.049486
130,115,Josh,Brownhill,Burnley,3,4.2,45,2812,0,16.4,74,a,Midfielder,2.368421,0.052632
503,426,João Filipe Iria,Santos Moutinho,Wolves,3,0.5,50,2526,0,14.6,73,a,Midfielder,2.600950,0.052019
51,50,Douglas Luiz,Soares de Paulo,Aston Villa,3,1.9,45,2781,0,16.0,72,a,Midfielder,2.330097,0.051780
6,7,Granit,Xhaka,Arsenal,3,0.9,50,2519,0,14.0,70,a,Midfielder,2.500992,0.050020
350,304,Jacob,Murphy,Newcastle,3,0.1,50,1617,0,13.8,69,a,Midfielder,3.840445,0.076809


In [28]:
cheap_def_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
506,429,Conor,Coady,Wolves,2,7.8,45,3303,0,23.6,106,a,Defender,2.888283,0.064184
13,14,Rob,Holding,Arsenal,2,2.4,45,2557,0,23.3,105,a,Defender,3.695737,0.082127
27,67,Ben,White,Arsenal,2,26.1,45,3192,0,23.1,104,a,Defender,2.932331,0.065163
125,110,Matthew,Lowton,Burnley,2,2.9,45,3060,0,22.9,103,a,Defender,3.029412,0.067320
240,185,Luke,Ayling,Leeds,2,12.0,45,3399,0,22.2,100,a,Defender,2.647838,0.058841
96,62,Joël,Veltman,Brighton,2,3.7,45,2280,0,21.3,96,a,Defender,3.789474,0.084211
400,343,Jan,Bednarek,Southampton,2,1.3,45,3100,0,20.9,94,a,Defender,2.729032,0.060645
185,488,Joachim,Andersen,Crystal Palace,2,0.5,45,2729,0,20.7,93,a,Defender,3.067058,0.068157
95,61,Adam,Webster,Brighton,2,0.6,45,2594,0,18.9,85,a,Defender,2.949113,0.065536
128,113,Charlie,Taylor,Burnley,2,0.7,45,2425,0,18.4,83,a,Defender,3.080412,0.068454


In [24]:
cheap_goal_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
165,146,Vicente,Guaita,Crystal Palace,1,5.6,45,3330,0,27.6,124,a,Goalkeeper,3.351351,0.074474
102,69,Robert,Sánchez,Brighton,1,25.3,45,2430,0,22.4,101,a,Goalkeeper,3.740741,0.083128
392,334,Alex,McCarthy,Southampton,1,1.7,45,2700,0,20.9,94,a,Goalkeeper,3.133333,0.069630
340,294,Karl,Darlow,Newcastle,1,1.3,45,2250,0,17.8,80,a,Goalkeeper,3.200000,0.071111
341,295,Martin,Dubravka,Newcastle,1,0.3,45,1170,0,10.7,48,d,Goalkeeper,3.692308,0.082051
99,65,Mathew,Ryan,Brighton,1,0.0,45,1260,0,8.2,37,u,Goalkeeper,2.642857,0.058730
391,333,Fraser,Forster,Southampton,1,1.2,45,720,0,6.7,30,a,Goalkeeper,3.750000,0.083333
238,183,Francisco,Casilla Cortés,Leeds,1,0.0,45,270,0,3.1,14,u,Goalkeeper,4.666667,0.103704
260,223,Adrián,San Miguel del Castillo,Liverpool,1,0.3,45,270,0,2.4,11,a,Goalkeeper,3.666667,0.081481
481,407,Darren,Randolph,West Ham,1,0.2,45,270,0,2.2,10,a,Goalkeeper,3.333333,0.074074
